# Анализ базы данных сервиса чтения книг (SQL)

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.  
  
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача в роли аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  
  
<b>В рамках анализа мы выполним следующие задачи:</b>

* посчитаем, сколько книг вышло после 1-го января 2000-го года;
* для каждой книги посчитаем количество обзоров и среднюю оценку;
* определим издательство, которое выпустило наибольшее число книг толще 50-ти страниц;
* определим автора с самой высокой средней оценкой книг (будем учитывать только книги с 50-ю и более оценками);
* посчитаем среднее количество обзоров от пользователей, которые поставили больше 50-ти оценок.

<b>План проекта:</b>

* Шаг 1. Загрузка данных, изучение общей информации
* Шаг 2. Подсчёт количества книг, вышедших после 1 января 2000 года
* Шаг 3. Подсчёт количества обзоров и средней оценки для каждой книги
* Шаг 4. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц
* Шаг 5. Определение автора с самой высокой средней оценкой книг
* Шаг 6. Определение среднего количества обзоров от пользователей, которые поставили больше 50 оценок
* Шаг 7. Общий вывод  

## Шаг 1. Загрузка данных, изучение общей информации

<b>Описание данных:</b>

<code><b>books</b></code> — таблица содержит данные о книгах;
* <code><b>book_id</b></code> — идентификатор книги;
* <code><b>author_id</b></code> — идентификатор автора;
* <code><b>title</b></code> — название книги;
* <code><b>num_pages</b></code> — количество страниц;
* <code><b>publication_date</b></code> — дата публикации книги;
* <code><b>publisher_id</b></code> — идентификатор издателя.

<code><b>authors</b></code> — таблица содержит данные об авторах;
* <code><b>author_id</b></code> — идентификатор автора;
* <code><b>author_id</b></code> — имя автора.

<code><b>publishers</b></code> — таблица содержит данные об издательствах;
* <code><b>publisher_id</b></code> — идентификатор издательства;
* <code><b>publisher</b></code> — название издательства.

<code><b>ratings</b></code> — таблица содержит данные о пользовательских оценках книг;
* <code><b>rating_id</b></code> — идентификатор оценки;
* <code><b>book_id</b></code> — идентификатор книги;
* <code><b>username</b></code> — имя пользователя, оставившего оценку;
* <code><b>rating</b></code> — оценка книги.

<code><b>reviews</b></code> — таблица содержит данные о пользовательских обзорах;
* <code><b>review_id</b></code> — идентификатор обзора;
* <code><b>book_id</b></code> — идентификатор книги;
* <code><b>username</b></code> — имя автора обзора;
* <code><b>text</b></code> — текст обзора.

<b>Схема данных:</b>

<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png">

Загрузим необходимые библиотеки и обеспечим доступ к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# создадим простейшую функцию проверки дубликатов
dict_data = {1:'books',
           2:'authors',
           3:'publishers',
           4:'ratings',
           5:'reviews'}

def print_duplicated(data, number):
    print('Обнаружено дубликатов ({}):'.format(dict_data[number]), data.duplicated().sum())
    print('От общей базы это составляет: ', round(((data.duplicated().sum() / len(data))*100), 2), '%', sep='')

### Загрузка данных таблицы books

Загрузим таблицу <code><b>books</b></code> и ознакомимся с данными.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>books</b></code> — таблица с данными о книгах
</div>

In [3]:
query = '''SELECT * FROM books'''
books = pd.io.sql.read_sql(query, con = engine)
display(books.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
display(books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

In [5]:
print_duplicated(books, 1)

Обнаружено дубликатов (books): 0
От общей базы это составляет: 0.0%


### Загрузка данных таблицы authors

Загрузим таблицу <code><b>authors</b></code> и ознакомимся с данными.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>authors</b></code> — таблица с данными об авторах
</div>

In [6]:
query = '''SELECT * FROM authors'''
authors = pd.io.sql.read_sql(query, con = engine) 
display(authors.head())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
display(authors.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

In [8]:
print_duplicated(authors, 2)

Обнаружено дубликатов (authors): 0
От общей базы это составляет: 0.0%


### Загрузка данных таблицы publishers

Загрузим таблицу <code><b>publishers</b></code> и ознакомимся с данными.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>publishers</b></code> — таблица с данными об издательствах
</div>

In [9]:
query = '''SELECT * FROM publishers'''
publishers = pd.io.sql.read_sql(query, con = engine) 
display(publishers.head())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
display(publishers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

In [11]:
print_duplicated(publishers, 3)

Обнаружено дубликатов (publishers): 0
От общей базы это составляет: 0.0%


### Загрузка данных таблицы ratings

Загрузим таблицу <code><b>ratings</b></code> и ознакомимся с данными.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>ratings</b></code> — таблица с данными о пользовательских оценках книг
</div>

In [12]:
query = '''SELECT * FROM ratings'''
ratings = pd.io.sql.read_sql(query, con = engine) 
display(ratings.head())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [13]:
display(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

In [14]:
print_duplicated(ratings, 4)

Обнаружено дубликатов (ratings): 0
От общей базы это составляет: 0.0%


### Загрузка данных таблицы reviews

Загрузим таблицу <code><b>reviews</b></code> и ознакомимся с данными.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>reviews</b></code> — таблица с данными о пользовательских обзорах
</div>

In [15]:
query = '''SELECT * FROM reviews'''
reviews = pd.io.sql.read_sql(query, con = engine) 
display(reviews.head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [16]:
display(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

In [17]:
print_duplicated(reviews, 5)

Обнаружено дубликатов (reviews): 0
От общей базы это составляет: 0.0%


### Выводы по первому шагу

<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font><br>
Выше изучив данные <code><b>books</b></code>, <code><b>authors</b></code>, <code><b>publishers</b></code>, <code><b>ratings</b></code> и <code><b>reviews</b></code>, обратил внимание на:
<br>

Отдельно по таблице <b>books</b>:
<br>

- в таблице 1000 строк и 6 столбцов;<br>
- типы данных корректные, за исключением столбца <code><b>publication_date</b></code> (но менять тип данных мы не будем т.к. работаем преимущественно в <code><b>SQL</b></code>);<br>
- пропуски отсутствуют;<br>
- дубликаты отсутствуют.
<br>

Отдельно по таблице <b>authors</b>:
<br>

- в таблице 636 строк и 2 столбца;<br>
- типы данных корректные;<br>
- пропуски отсутствуют;<br>
- дубликаты отсутствуют.
<br>    
    
Отдельно по таблице <b>publishers</b>:
<br>

- в таблице 340 строк и 2 столбца;<br>
- типы данных корректные;<br>
- пропуски отсутствуют;<br>
- дубликаты отсутствуют.
<br>   

Отдельно по таблице <b>ratings</b>:
<br>

- в таблице 6456 строк и 4 столбца;<br>
- типы данных корректные;<br>
- пропуски отсутствуют;<br>
- дубликаты отсутствуют.
<br>     
    
Отдельно по таблице <b>reviews</b>:
<br>

- в таблице 2793 строк и 4 столбца;<br>
- типы данных корректные;<br>
- пропуски отсутствуют;<br>
- дубликаты отсутствуют.
<br>    

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 2. Подсчёт количества книг, вышедших после 1 января 2000 года

Посчитаем, сколько книг вышло после 1 января 2000 года.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>task1</b></code> — количество книг вышедших после 1 января 2000 года
</div>

In [18]:
query = '''SELECT COUNT(book_id) AS number_of_books
           FROM books
           WHERE publication_date > '2000-01-01'
           '''
task1 = pd.io.sql.read_sql(query, con = engine) 
display(task1)

,number_of_books
0,819


<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
Всего после 1 января 2000 года вышло 819 книг.<br>

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 3. Подсчёт количества обзоров и средней оценки для каждой книги

Для каждой книги посчитаем количество обзоров и среднюю оценку.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>task2</b></code> — количество обзоров и средние оценки для каждой книги
</div>

In [19]:
query = '''
         SELECT
               books.book_id,
               books.title AS name_book,
               COUNT(DISTINCT reviews.review_id) AS count_review,
               AVG(ratings.rating) AS avg_rating
         FROM  books
               LEFT JOIN reviews ON reviews.book_id = books.book_id
               LEFT JOIN ratings ON ratings.book_id = books.book_id
         GROUP BY
               books.book_id
         ORDER BY
               count_review DESC
               
        '''
task2= pd.io.sql.read_sql(query, con = engine)
display(task2)

,book_id,name_book,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
Лидером по количеству отзывов стало произведение <code><b>Twilight (Twilight #1)</b></code>.<br>

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 4. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>task3</b></code> — издательство, которое выпустило наибольшее число книг толще 50 страниц
</div>

In [20]:
query = '''SELECT publisher,
                  COUNT(book_id) AS number_of_books
           FROM publishers p
           JOIN books b ON b.publisher_id = p.publisher_id
           GROUP BY publisher
           HAVING MIN(num_pages) > 50
           ORDER BY number_of_books DESC
           LIMIT 1
           '''
task3 = pd.io.sql.read_sql(query, con = engine) 
display(task3)

,publisher,number_of_books
0,Penguin Books,42


<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
Издательство, выпустившее наибольшее число книг толще 50 страниц - <code><b>Penguin Books</b></code> с количеством выпущенных книг равным 42.<br>

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 5. Определение автора с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг (учтём только книги с 50 и более оценками).

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>task4</b></code> — автор с самой высокой средней оценкой книг
</div>

In [21]:
query = '''SELECT authors.author,
                  top.avg AS average_rating
           FROM authors
           RIGHT JOIN (SELECT author_id,
                              AVG(rating_avg)
                       FROM books
                       LEFT JOIN (SELECT AVG(rating) AS rating_avg, 
                                         book_id 
                                  FROM ratings
                                  GROUP BY book_id) AS ratings ON books.book_id = ratings.book_id
                       WHERE books.book_id IN (SELECT books.book_id
                                               FROM books
                                               LEFT JOIN ratings ON books.book_id = ratings.book_id
                                               GROUP BY books.book_id
                                               HAVING COUNT(rating_id) >= 50)
                       GROUP BY author_id
                       ORDER BY avg DESC
                       LIMIT 1) AS top ON authors.author_id = top.author_id
           '''
task4 = pd.io.sql.read_sql(query, con = engine) 
display(task4)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
Автор с самой высокой средней оценкой книг - <code><b>J.K. Rowling/Mary GrandPré</b></code> (для книг с количеством оценок больше 50).<br>

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 6. Определение среднего количества обзоров от пользователей, которые поставили больше 50 оценок

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">
<code><b>task5</b></code> — среднее количество обзоров от пользователей, которые поставили больше 50 оценок
</div>

In [22]:
query = '''SELECT AVG((COUNT)) AS avg_reviews
           FROM
               (SELECT COUNT(text)
                FROM reviews
                FULL OUTER JOIN (SELECT username,
                                        COUNT(rating_id) as cnt
                                 FROM ratings
                                 GROUP BY username) AS cnt_rating
                ON cnt_rating.username = reviews.username
                WHERE cnt > 50
                GROUP BY reviews.username) as count;                                  
                '''
task5 = pd.io.sql.read_sql(query, con = engine) 
display(task5)

,avg_reviews
0,24.333333


<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
Среднее количество отзывов от пользователей, поставивших больше 50 оценок - 24,3.<br>

</div>

<hr style="border: 2px solid indigo;" />

## Шаг 7. Общий вывод

<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>ВЫВОД:</b></font>
<br>
    
- После 1 января 2000 года вышло 819 книг;<br>
- Для книги <code><b>Twilight</b></code> вышло 7 обзоров при рейтинге книги 3.7, в то время как для книги <code><b>The Hobbit or There and Back Again</b></code> при рейтинге 4.1 вышло 6 обзоров;<br>
- Издательство, выпустившее наибольшее число книг толще 50 страниц: <code><b>Penguin Books</b></code> 42 книги;<br>
- Автор с самой высокой средней оценкой книг: <code><b>J.K. Rowling/Mary GrandPré</b></code> с рейтингом 4.28;<br>
- Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.3.
<br>

</div>

<div style="border-radius: 15px; border: 3px solid green; padding: 15px;">
<font color='green'><b>РЕКОМЕНДАЦИИ:</b></font>
<br>
    
- Продвигать книги имеющие достаточно высокий рейтинг или большое количество обзоров;<br>
- Активных пользователей выделить на фоне остальных и стимулировать оставлять еще больше обзоров на прочитанные книги;<br>
- Заключить с издательством, выпускающее больше всего книг (толще 50 листов) договор о предоставление новых книг в наш сервис в числе первых.
<br>

</div>

<hr style="border: 2px solid indigo;" />